In [1]:
import torch
print(torch.cuda.is_available() , torch.__version__)

False 2.2.0+cpu


In [1]:
# %% test a specific model
from src.interface import ModelTrainer
ModelTrainer.main(stage = 0 , resume = 0 , checkname= 1)

24-05-03 10:47:41|MOD:display     |: Model Specifics:
24-05-03 10:47:41|MOD:display     |: Start Process [Data] at Fri May  3 10:47:41 2024!


--Process Queue : Data + Fit + Test
--Model_name is set to gru_day_ShortTest!
Callback : DynamicDataLink() , assign / unlink dynamic data in tra networks
Callback : ResetOptimizer(num_reset=2,trigger=40,recover_level=1.0,speedup2x=True) , reset optimizer on some epoch (can speedup scheduler)
Callback : CallbackTimer(verbosity=10) , record time cost of callback hooks
Callback : EarlyStoppage(patience=20) , stop fitting when validation score cease to improve
Callback : ValidationConverge(patience=5,eps=1e-05) , stop fitting when valid_score converge
Callback : EarlyExitRetrain(earliest=20,max_attempt=4,lr_multiplier=[1, 0.1, 10, 0.01, 100, 1]) , retrain with new lr if fitting stopped too early
Callback : NanLossRetrain(max_attempt=4) , retrain if fitting encounters nan loss
Callback : BatchDisplay(verbosity=10) , display batch progress bar
Callback : StatusDisplay(verbosity=10) , display epoch / event information
{'random_seed': None,
 'model_name': 'gru_day_ShortTest',
 'model_module': 

24-05-03 10:47:43|MOD:display     |: Finish Process [Data], Cost 1.6 Secs
24-05-03 10:47:43|MOD:display     |: Start Process [Fit] at Fri May  3 10:47:43 2024!


Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}


  0%|          | 0/10 [00:00<?, ?it/s]

score function of [spearman] calculated and success!
loss function of [pearson] calculated and success!


Valid Ep#  0 score : -0.04784: 100%|██████████| 10/10 [00:02<00:00,  4.47it/s]
FirstBite Ep#  0 : loss  1.00177, train-0.00045, valid-0.04784, best-0.0478, lr1.3e-03
Valid Ep#  1 score : 0.11754: 100%|██████████| 10/10 [00:02<00:00,  4.22it/s]
FirstBite Ep#  1 : loss  0.96937, train 0.03194, valid 0.11754, best 0.1175, lr2.5e-03
Valid Ep#  2 score : 0.07742: 100%|██████████| 10/10 [00:02<00:00,  4.09it/s]
FirstBite Ep#  2 : loss  0.93689, train 0.06355, valid 0.07742, best 0.1175, lr3.8e-03
Valid Ep#  3 score : 0.07735: 100%|██████████| 10/10 [00:02<00:00,  4.62it/s]
FirstBite Ep#  3 : loss  0.91731, train 0.08939, valid 0.07735, best 0.1175, lr5.0e-03
24-05-03 10:49:40|MOD:display     |: gru_day_ShortTest #0 @20170103|FirstBite Ep#  4 Max Epoch|Train 0.0894 Valid 0.0773 BestVal 0.1175|Cost  1.9Min, 23.1Sec/Ep
24-05-03 10:49:40|MOD:display     |: Finish Process [Fit], Cost 0.0 Hours, 2.0 Min/model, 29.3 Sec/Epoch
24-05-03 10:49:40|MOD:display     |: Start Process [Test] at Fri May  3 1

                    hook_name  num_calls  total_time   avg_time
8              on_train_batch         40   86.811861   2.170297
23              on_test_batch        105   27.235953   0.259390
17           on_fit_model_end          1   19.183584  19.183584
13        on_validation_batch         40    9.140732   0.228518
4          on_fit_model_start          1    1.647796   1.647796
20        on_test_model_start          1    0.590456   0.590456
24          on_test_batch_end        105    0.093101   0.000887
21   on_test_model_type_start          3    0.035427   0.011809
14    on_validation_batch_end         40    0.034552   0.000864
9          on_train_batch_end         40    0.030182   0.000755
16           on_fit_epoch_end          4    0.028284   0.007071
15    on_validation_epoch_end          4    0.022986   0.005746
27                on_test_end          1    0.015959   0.015959
10         on_train_epoch_end          4    0.013140   0.003285
11  on_validation_epoch_start          4

In [ ]:
from src.interface import ModelTrainer
ModelTrainer.main(stage = 0 , resume = 0 , checkname= 1)

In [ ]:
# prepare data
from src.interface import DataModule
DataModule.prepare_data()